<a href="https://colab.research.google.com/github/ph20d204/Instrument_Automation/blob/main/NIDAQmx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automation of Pulser Streamer
> Snigdhadev and Atanu
>> to automate the NIDAQmx USB-6361

In [ ]:
## To do a Edge_counting operation

# loading the local NIDAQmx system
system = nidaqmx.system.System.local()
DAQ_device = system.devices['Dev1']

# Creating a list of all the available counters
counter_names = [ci.name for ci in DAQ_device.ci_physical_chans]
print(counter_names)
print([co.name for co in DAQ_device.co_physical_chans])   #includes the '[device]/freqout' output, which is not a counter

# Creating a task for the counter channel and a task for a 'dummy' digital input channel to start the digital input Sample Clock(mandatory).
with nidaqmx.Task() as read_task, nidaqmx.Task() as samp_clk_task:
    # create a digital input channel on 'port0' of 'Dev1'
    samp_clk_task.di_channels.add_di_chan('Dev1/port0')
        """
        Note that port2 of a DAQ device does not support buffered operations, so here port port0 is used. Additionally, the
        line_grouping Arg (1 channel for all lines or 1 channel per line) does not matter because this is a 'dummy' task.
        """
    sampling_rate = 100     #100 Hz
    samp_clk_task.timing.cfg_samp_clk_timing(rate=sampling_rate, sample_mode=AcquisitionType.CONTINUOUS)

    # The hardware will now be in the Commit state and only transition to the State state to run the task.
    samp_clk_task.control(TaskMode.TASK_COMMIT)


    # Creating a counter input channel using 'ctr0' on 'Dev1' to count rising digital edges, counting up from initial_count
    read_task.ci_channels.add_ci_count_edges_chan('Dev1/ctr0',edge=Edge.RISING,initial_count=0,count_direction=CountDirection.COUNT_UP)

    # Setting the input terminal of the counter input channel on which the counter receives the signal on which it counts edges
    read_task.ci_channels.all.ci_count_edges_term = '/Dev1/PFI5'

       """
       When specifying the name of a terminal, all external terminals - as defined by NI-DAQmx - must include a leading
       '/' in its string. An external terminal is any terminal that can be routed internally from one channel to another or from
       DAQ to another; examples include: PFI lines, Sample Clocks, physical analog channels, physical digital channels, the
       output of a physical counter, etc. All external terminals can be 'exported' using task.export_signals.export_signal(*args).
       NI-DAQmx recognized devices do not include a leading '/' in their string name because they are not terminals.
       """

    # The hardware will not be optimized if a different sampling rate is set than the true rate.
    # Additionally, setting the counter to readout its count to the buffer on the rising edge of the Sample Clock signal.
    """ max counter sampling rate allowed: 100e6 (i.e. 100MHz)"""
    read_task.timing.cfg_samp_clk_timing(sampling_rate, source='/Dev1/di/SampleClock', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
        """
        Other optional Arg is 'samps_per_chan': if ** sample_mode** is **CONTINUOUS_SAMPLES**, NI-DAQmx uses this value to
        determine the buffer size. 'cfg_samp_clk_timing' returns an error if the specified value is negative.
        """
    # Setting the buffer size of the counter, such that, given the sampling rate at which the counter reads out its current value
    # to the buffer, it will give two minutes of samples before the buffer overflows.
    read_task.in_stream.input_buf_size = 12000

    # Creating an arm start trigger for the counter so that it is synced with the digital input Sample Clock and only starts
    # counting when the first Sample Clock tick is detected. This prevents the necessity of throwing out the first sample in the counter buffer.
    read_task.triggers.arm_start_trigger.trig_type = TriggerType.DIGITAL_EDGE
    read_task.triggers.arm_start_trigger.dig_edge_edge = Edge.RISING
    read_task.triggers.arm_start_trigger.dig_edge_src = '/Dev1/di/SampleClock'

    # Creating a counter reader to read from the counter InStream
    reader = CounterReader(read_task.in_stream)
    # Starting the tasks to begin data acquisition; as the arm start trigger of the counter was set, the tasks will be synced irrespective of which starts first
    samp_clk_task.start()
    read_task.start()

    # Creating a data buffer for the counter stream reader
    data_array = numpy.zeros(12000, dtype=numpy.uint32)
    # Reading all samples from the counter buffer to the system memory buffer data_array; if the buffer is not large enough, it will raise an error
    reader.read_many_sample_uint32(data_array, number_of_samples_per_channel=READ_ALL_AVAILABLE)

    # NOTE: the below calls do not need to be used at the end of a code block when a 'with' block is implemented for taskcreation; this is handled automatically.
    # The below lines are for illustration purposes.
    read_task.stop()
    read_task.start()
    read_task.close()

## nidaqmx.system

In [ ]:
import nidaqmx.system.system as system
import collections.abc as collections
from collections.abc import Sequence

In [ ]:
system.System().devices[0], system.System.local().devices[0], system.System.devices

(Device(name=Dev1), Device(name=Dev1), <property at 0x1a859593380>)

In [ ]:
system.System.local().devices['Dev1'].terminals

['/Dev1/PFI0',
 '/Dev1/PFI1',
 '/Dev1/PFI2',
 '/Dev1/PFI3',
 '/Dev1/PFI4',
 '/Dev1/PFI5',
 '/Dev1/PFI6',
 '/Dev1/PFI7',
 '/Dev1/PFI8',
 '/Dev1/PFI9',
 '/Dev1/PFI10',
 '/Dev1/PFI11',
 '/Dev1/PFI12',
 '/Dev1/PFI13',
 '/Dev1/PFI14',
 '/Dev1/PFI15',
 '/Dev1/APFI0',
 '/Dev1/20MHzTimebase',
 '/Dev1/100MHzTimebase',
 '/Dev1/10MHzRefClock',
 '/Dev1/ai/ReferenceTrigger',
 '/Dev1/ai/SampleClock',
 '/Dev1/ai/StartTrigger',
 '/Dev1/ao/SampleClock',
 '/Dev1/ao/StartTrigger',
 '/Dev1/di/SampleClock',
 '/Dev1/di/StartTrigger',
 '/Dev1/di/ReferenceTrigger',
 '/Dev1/do/SampleClock',
 '/Dev1/do/StartTrigger',
 '/Dev1/ChangeDetectionEvent',
 '/Dev1/WatchdogExpiredEvent',
 '/Dev1/WatchdogExpirationTrigger',
 '/Dev1/AnalogComparisonEvent',
 '/Dev1/ai/ConvertClock',
 '/Dev1/ai/ConvertClockTimebase',
 '/Dev1/ai/HoldCompleteEvent',
 '/Dev1/ai/PauseTrigger',
 '/Dev1/ai/SampleClockTimebase',
 '/Dev1/ao/PauseTrigger',
 '/Dev1/ao/SampleClockTimebase',
 '/Dev1/di/SampleClockTimebase',
 '/Dev1/di/PauseTrigger',
 '/

## nidaqmx.stream_readers

In [ ]:
import nidaqmx.stream_readers as reader

In [ ]:
reader.AnalogSingleChannelReader.mro()

[nidaqmx.stream_readers.AnalogSingleChannelReader,
 nidaqmx.stream_readers.ChannelReaderBase,
 object]

## nidaqmx.scale

In [ ]:
import nidaqmx.scale as scale

In [ ]:
print(inspect.getsource(scale.ScaleType))

class ScaleType(Enum):
    LINEAR = 10447  #: Scale values by using the equation y=mx+b, where x is a prescaled value and y is a scaled value.
    MAP_RANGES = 10448  #: Scale values proportionally from a range of pre-scaled values to a range of scaled values.
    POLYNOMIAL = 10449  #: Scale values by using an Nth order polynomial equation.
    TABLE = 10450  #: Map a list of pre-scaled values to a list of corresponding scaled values, with all other values scaled proportionally.
    NONE = 10230  #: Do not scale electrical values to physical units.
    TWO_POINT_LINEAR = 15898  #: You provide two pairs of electrical values and their corresponding physical values. NI-DAQmx uses those values to calculate the slope and y-intercept of a linear equation and uses that equation to scale electrical values to physical values.

